In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

load_dotenv()

True

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


loader = DirectoryLoader("/home/jinzy/work/automation/ask_gpt/documents", glob="**/*.pdf", show_progress=True, use_multithreading=True)

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 400,
    chunk_overlap  = 30,
    length_function = len,
)
txt_docs = loader.load_and_split(text_splitter=text_splitter)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 224/224 [14:08<00:00,  3.79s/it]


In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.llms import GPT4All
from langchain import HuggingFaceHub, LLMChain

# initialize Hub LLM
# llm = HuggingFaceHub(repo_id='THUDM/chatglm-6b', model_kwargs={"trust_remote_code": True})
# llm = HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":1e-10})
# llm = ChatGLMService()
# llm.load_model()

embeddings = OpenAIEmbeddings()
# embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", 
#                                            model_kwargs={"device": "cuda"})
persist_directory = 'rpmv2'
txt_docsearch = Chroma.from_documents(txt_docs, embeddings, persist_directory=persist_directory)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5, max_tokens=3500)
# llm = GPT4All(model="/home/jinzy/Downloads/ggml-gpt4all-j-v1.3-groovy.bin", n_ctx=512, n_threads=8)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 3c4b3dc691296b6e6980815c4b48ddd2 in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 3c4b3dc691296b6e6980815c4b48ddd2 in your message.)",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 3c4b3dc69129

In [4]:
txt_docsearch.persist()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [7]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5, max_tokens=3500)

In [8]:
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
  ChatPromptTemplate,
  SystemMessagePromptTemplate,
  HumanMessagePromptTemplate
)
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory


rewritte_template = """根据聊天记录和问题，把问题重写为一个独立的、完整的问题。
聊天记录:
{chat_history}
问题: {question}
独立问题:"""

# 初始化 prompt 对象
prompt = ChatPromptTemplate.from_template(rewritte_template)
# 初始化问答链
question_generator = LLMChain(llm=llm, prompt=prompt)

refine_template = """请检查下述检索结果是否可以用于更新回复内容。
检索结果:
{context_str}

如果这个检索结果包含了有用的信息，请根据它和原始回复内容重新生成更好的回复。如果没有帮助，就返回原始回复内容。
新的回复："""
# (
#     "We have the opportunity to refine the existing answer"
#     "(only if needed) with some more context below.\n"
#     "------------\n"
#     "{context_str}\n"
#     "------------\n"
#     "Given the new context, refine the original answer to better "
#     "answer the question. "
#     "If the context isn't useful, return the original answer."
# )
refine_prompt = ChatPromptTemplate.from_template(refine_template)
doc_chain = load_qa_chain(llm, chain_type="refine", refine_prompt =refine_prompt)

summary_template = """根据聊天内容增量式生成摘要，在原始摘要上附加新的摘要。
EXAMPLE
当前摘要: 
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

新的聊天内容:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

新的摘要:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

当前摘要:
{summary}

新的聊天内容:
{new_lines}

新的摘要:
"""

summary_prompt = PromptTemplate(input_variables=["summary", "new_lines"], template=summary_template)

# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory = ConversationSummaryBufferMemory(llm=llm, prompt = summary_prompt,
        max_token_limit=1000, memory_key="chat_history", return_messages=True)
chain = ConversationalRetrievalChain(
    retriever=txt_docsearch.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    memory=memory
)
# chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=txt_docsearch.as_retriever(), 
#                                               chain_type='refine', memory=memory)

In [14]:
query = "请用简单的中文，尽可能详细地解释一下RPM产品中的任务挖掘功能"
result = chain({"question": query})
print(result)

{'question': '请用简单的中文，尽可能详细地解释一下RPM产品中的任务挖掘功能', 'chat_history': [HumanMessage(content='请用简单的语言尽可能详细地解释一下RPM产品中的任务挖掘功能', additional_kwargs={}, example=False), AIMessage(content='Robotic Process Mining (RPM) is a new family of techniques that aims to assist RPA developers and analysts in the early stages of the RPA lifecycle. RPM helps analysts draw a systematic inventory of tasks that can be automated with RPA and synthesizes executable specifications of such tasks, which can be used as a starting point.', additional_kwargs={}, example=False), HumanMessage(content='请给出一个RPM产品的实际案例', additional_kwargs={}, example=False), AIMessage(content='这篇论文所讨论的问题可以放在Robotic Process Mining (RPM) [2]的更广泛背景下来看。RPM是一系列的方法，用于发现员工在日常工作中执行的重复例程，并将这些例程转化为软件脚本以模拟它们的执行。RPM流程的第一步是记录一个或多个工人与一个或多个系统之间的交互。这些信息可以帮助我们更好地理解如何自动化这些例程，提高工作效率。', additional_kwargs={}, example=False), HumanMessage(content='请介绍一个RPM产品的实际案例', additional_kwargs={}, example=False), AIMessage(content='Robotic Process Mining (RPM) 是一种通过分析一个或多个工

In [10]:
query = "请用简单的中文，给出一个RPM产品的实际案例"
#result = chain({"question": query, "chat_history": chat_history})
result = chain({"question": query})
#print(result)
print(result['answer'])

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 8776afb7f82b9d963ba925e6bd78e18e in your message.).


这篇论文所讨论的问题可以放在Robotic Process Mining (RPM) [2]的更广泛背景下来看。RPM是一系列的方法，用于发现员工在日常工作中执行的重复例程，并将这些例程转化为软件脚本以模拟它们的执行。RPM流程的第一步是记录一个或多个工人与一个或多个系统之间的交互。这些信息可以帮助我们更好地理解如何自动化这些例程，提高工作效率。


In [11]:
query = "请用简单的中文，介绍一个RPM产品的实际案例"
#result = chain({"question": query, "chat_history": chat_history})
result = chain({"question": query})

print(result['answer'])

Robotic Process Mining (RPM) 是一种通过分析一个或多个工作人员与一个或多个软件应用程序之间的交互来发现可以使用 RPA 技术自动化的重复例程的技术。通常，RPM 技术的输入为用户交互日志（UI 日志）。这些 UI 日志是在执行业务流程中的一个或多个任务时记录的。


In [12]:
result

{'question': '请介绍一个RPM产品的实际案例',
 'chat_history': [HumanMessage(content='请用简单的语言尽可能详细地解释一下RPM产品中的任务挖掘功能', additional_kwargs={}, example=False),
  AIMessage(content='Robotic Process Mining (RPM) is a new family of techniques that aims to assist RPA developers and analysts in the early stages of the RPA lifecycle. RPM helps analysts draw a systematic inventory of tasks that can be automated with RPA and synthesizes executable specifications of such tasks, which can be used as a starting point.', additional_kwargs={}, example=False),
  HumanMessage(content='请给出一个RPM产品的实际案例', additional_kwargs={}, example=False),
  AIMessage(content='这篇论文所讨论的问题可以放在Robotic Process Mining (RPM) [2]的更广泛背景下来看。RPM是一系列的方法，用于发现员工在日常工作中执行的重复例程，并将这些例程转化为软件脚本以模拟它们的执行。RPM流程的第一步是记录一个或多个工人与一个或多个系统之间的交互。这些信息可以帮助我们更好地理解如何自动化这些例程，提高工作效率。', additional_kwargs={}, example=False),
  HumanMessage(content='请介绍一个RPM产品的实际案例', additional_kwargs={}, example=False),
  AIMessage(content='Robotic Process Mining (RPM) 是一种通过分析一个或多个工作人员与一

In [ ]:
# query = "请介绍一个RPM产品的实际案例"
# #result = chain({"question": query, "chat_history": chat_history})
# result = chain({"question": query + ' 请用中文回答。'})

# print(result['answer'])

In [ ]:
# result

In [ ]:
# chain.memory

In [ ]:
# from typing import List, Optional
# from langchain.llms.base import LLM
# from langchain.llms.utils import enforce_stop_tokens
# from transformers import AutoModel, AutoTokenizer


# class ChatGLMService(LLM):
#     max_token: int = 10000
#     temperature: float = 0.1
#     top_p = 0.9
#     history = []
#     tokenizer: object = None
#     model: object = None

#     def __init__(self):
#         super().__init__()

#     @property
#     def _llm_type(self) -> str:
#         return "ChatGLM"

#     def _call(self,
#               prompt: str,
#               stop: Optional[List[str]] = None) -> str:
#         response, _ = self.model.chat(
#             self.tokenizer,
#             prompt,
#             history=self.history,
#             max_length=self.max_token,
#             temperature=self.temperature,
#         )
#         if stop is not None:
#             response = enforce_stop_tokens(response, stop)
#         self.history = self.history + [[None, response]]
#         return response

#     def load_model(self, model_name_or_path: str = "THUDM/chatglm-6b"):
#         self.tokenizer = AutoTokenizer.from_pretrained(
#             model_name_or_path,
#             trust_remote_code=True
#         )
#         print('加载chatglm-6b', model_name_or_path)
#         # 这里使用的是 量化后的模型int4
#         self.model = AutoModel.from_pretrained(model_name_or_path, trust_remote_code=True).float().half().cuda()
#         #.half().quantize(4).cuda()
#         #.half().cuda()
#         self.model=self.model.eval()

In [ ]:
# import chromadb
# import chromadb.config

# persist_directory = 'rpmv2'
# txt_docsearch._client_settings = chromadb.config.Settings(
#                     chroma_db_impl="duckdb+parquet", persist_directory = persist_directory
#                 )
# txt_docsearch._client = chromadb.Client(txt_docsearch._client_settings)
# txt_docsearch._persist_directory = persist_directory
# txt_docsearch.persist()